In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import skimage
from skimage import io, morphology
import cv2
import numpy as np


def resizeImage(img, w=0, h=0):
    return cv2.resize(img, (800 + w * 20, 800 + h * 20), interpolation=cv2.INTER_LINEAR)


def maskImage(img2):
    #alpha- Contrast control (1.0-3.0)
    #beta- Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(img2, alpha=2.5 , beta=2)
    adjusted[adjusted < 180] = 0
    lower = np.array([160, 160, 160])
    upper = np.array([255, 255, 255])
    thresh = cv2.inRange(adjusted, lower, upper)
    adjusted[thresh == 255] = 0
    adjusted[thresh == 0] = 255
    
    gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
        
    kernel_erod = np.ones((1,1),np.uint8)
    kernel_dilate = np.ones((3,3),np.uint8)

    dil =cv2.erode(gray,kernel_erod,iterations = 1)
    mask = cv2.dilate(dil ,kernel_dilate,iterations = 2)

    return(mask) 


def cropObject(obj, img):
    global object_counter
    object_counter += 1
    x, y, w, h = cv2.boundingRect(obj)
    
    src= img.copy()
    center, size, theta = cv2.minAreaRect(obj)
    shape=(w,h)
    # Convert to int 
    center, size = tuple(map(int, center)), tuple(map(int, shape))
    # Get rotation matrix for rectangle
    if abs(abs(theta)-45) > 10:
        M = cv2.getRotationMatrix2D( center, theta+45, 1)
    # Perform rotation on src image
        dst = cv2.warpAffine(src, M, src.shape[:2])
        cropped_contour = cv2.getRectSubPix(dst, shape, center)
    else: 
        cropped_contour= img[y:y+h, x:x+w]
    
    #cropped_contour = img[y:y + h, x:x + w]
    image_name = "anaconda3\Lib\site-packages\skimage\data\TicTacToe\output_tictactoe" + str(object_counter) + ".jpg"
    cv2.imwrite(image_name, cropped_contour)
    readimage = cv2.imread(image_name)
    return readimage


def findingBoard(contours):
    max = 0
    for i in range(len(contours)):
        if len(contours[i]) > max:
            max  = len(contours[i])

    for i in range(len(contours)):
        if len(contours[i]) == max:
            return contours[i]

    return 0

def findingContours(image):
    # image = cv2.imread(image_name)
    contours, hierarchy = cv2.findContours(image=image, mode=cv2.RETR_LIST,
                                           method=cv2.CHAIN_APPROX_NONE)
    return contours, hierarchy


def sieveContours(contours, hierarchy):
    contour_list = []

    for i in range(len(contours)):
        if len(contours[i]) > 200 and hierarchy[0][i][2] == -1:
            contour_list.append(contours[i])

    return contour_list


def circlesCrosses(contour_list):
    circles = []
    crosses = []
    for cnt in contour_list:
        area = cv2.contourArea(cnt)
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area
        if solidity > 0.9:
            circles.append(cnt)
        else:
            crosses.append(cnt)

    return circles, crosses


def findCenter(contours):
    centers = []

    for cnt in contours:
        M = cv2.moments(cnt)

        # Centroid
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        centers.append([cx, cy])

    return centers


def nameContour(centers, type, image):
    for i in range(len(centers)):
        cv2.putText(image, type, (centers[i][0], centers[i][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 5, 2)


def emptyCells(centers, middle_points, centers_X):
    cells = [1,2,3,4,5,6,7,8,9]
    xs = []
    os = []
    for i in range(len(centers)):
        if centers[i][0] < middle_points[0][0]:
            if centers[i][1] < middle_points[0][1]:
                if 1 in cells:
                    cells.remove(1)
                    if centers[i] in centers_X:
                        xs.append(1)
                    else:
                        os.append(1)
            elif centers[i][1] < middle_points[2][1]:
                if 4 in cells:
                    cells.remove(4)
                    if centers[i] in centers_X:
                        xs.append(4)
                    else:
                        os.append(4)
            else:
                if 7 in cells:
                    cells.remove(7)
                    if centers[i] in centers_X:
                        xs.append(7)
                    else:
                        os.append(7)
        elif centers[i][0] < middle_points[1][0]:
            if centers[i][1] < middle_points[0][1]:
                if 2 in cells:
                    cells.remove(2)
                    if centers[i] in centers_X:
                        xs.append(2)
                    else:
                        os.append(2)
            elif centers[i][1] < middle_points[2][1]:
                if 5 in cells:
                    cells.remove(5)
                    if centers[i] in centers_X:
                        xs.append(5)
                    else:
                        os.append(5)
            else:
                if 8 in cells:
                    cells.remove(8)
                    if centers[i] in centers_X:
                        xs.append(8)
                    else:
                        os.append(8)
        else:
            if centers[i][1] < middle_points[0][1]:
                if 3 in cells:
                    cells.remove(3)
                    if centers[i] in centers_X:
                        xs.append(3)
                    else:
                        os.append(3)
            elif centers[i][1] < middle_points[2][1]:
                if 6 in cells:
                    cells.remove(6)
                    if centers[i] in centers_X:
                        xs.append(6)
                    else:
                        os.append(6)
            else:
                if 9 in cells:
                    cells.remove(9)
                    if centers[i] in centers_X:
                        xs.append(9)
                    else:
                        os.append(9)
    return cells, xs, os

def emptyCenters(empty, mid_points, minmax):
    centers = []
    for i in range(len(empty)):
        if empty[i] == 1:
            x = minmax[0] + (mid_points[0][0] - minmax[0]) // 2
            y = minmax[2] + (mid_points[0][1] - minmax[2]) // 2
        elif empty[i] == 2:
            x = mid_points[0][0] + (mid_points[1][0] - mid_points[0][0]) // 2
            y = minmax[2] + (mid_points[0][1] - minmax[2]) // 2
        elif empty[i] == 3:
            x = mid_points[1][0] + (minmax[1] - mid_points[1][0]) // 2
            y = minmax[2] + (mid_points[0][1] - minmax[2]) // 2
        elif empty[i] == 4:
            x = minmax[0] + (mid_points[0][0] - minmax[0]) // 2
            y = mid_points[0][1] + (mid_points[2][1] - mid_points[0][1]) // 2
        elif empty[i] == 5:
            x = mid_points[0][0] + (mid_points[1][0] - mid_points[0][0]) // 2
            y = mid_points[0][1] + (mid_points[2][1] - mid_points[0][1]) // 2
        elif empty[i] == 6:
            x = mid_points[1][0] + (minmax[1] - mid_points[1][0]) // 2
            y = mid_points[0][1] + (mid_points[2][1] - mid_points[0][1]) // 2
        elif empty[i] == 7:
            x = minmax[0] + (mid_points[0][0] - minmax[0]) // 2
            y = mid_points[2][1] + (minmax[3] - mid_points[2][1]) // 2
        elif empty[i] == 8:
            x = mid_points[0][0] + (mid_points[1][0] - mid_points[0][0]) // 2
            y = mid_points[2][1] + (minmax[3] - mid_points[2][1]) // 2
        elif empty[i] == 9:
            x = mid_points[1][0] + (minmax[1] - mid_points[1][0]) // 2
            y = mid_points[2][1] + (minmax[3] - mid_points[2][1]) // 2
        centers.append([x,y])
    return centers

def gameState(xs, os):
    global board
    global play_num

    game = ["-","-","-","-","-","-","-","-","-"]
    for i in range(len(xs)):
        pos = xs[i] - 1
        game[pos] = "X"
    for i in range(len(os)):
        pos = os[i] - 1
        game[pos] = "O"
    print("Stan rozgrywki TicTacToe nr "+ str(play_num+1))
    print(game[0]," | ", game[1], " | ", game[2])
    print("---------------")
    print(game[3], " | ", game[4], " | ", game[5])
    print("---------------")
    print(game[6], " | ", game[7], " | ", game[8])
    
    if str(game[0]) == "O" and str(game[4])=="O" and str(game[8])== "O":
        print('Wygrywa O!')
        print("\n")      
    elif str(game[0]) == "X" and str(game[4])=="X" and str(game[8])== "X":
        print('Wygrywa X!')
        print("\n")

        
    elif str(game[0]) == "X" and str(game[1])=="X" and str(game[3])== "X":
        print('X wins')
        print("\n")
    elif str(game[0]) == "O" and str(game[1])=="O" and str(game[3])== "O":
        print('O wins')
        print("\n")

        
    elif str(game[3]) == "X" and str(game[4])=="X" and str(game[5])== "X":
        print('X wins')
        print("\n")
    elif str(game[3]) == "O" and str(game[4])=="O" and str(game[5])== "O":
        print('O wins')
        print("\n")


    elif str(game[6]) == "X" and str(game[7])=="X" and str(game[8])== "X":
        print('X wins')
        print("\n")
    elif str(game[6]) == "O" and str(game[7])=="O" and str(game[8])== "O":
        print('O wins')
        print("\n")

        
    elif str(game[6]) == "X" and str(game[4])=="X" and str(game[2])== "X":
        print('X wins')
        print("\n")
    elif str(game[6]) == "O" and str(game[4])=="O" and str(game[2])== "O":
        print('O wins')
        print("\n")

        
    elif str(game[0]) == "X" and str(game[3])=="X" and str(game[6])== "X":
        print('X wins')
        print("\n")
    elif str(game[0]) == "O" and str(game[3])=="O" and str(game[6])== "O":
        print('O wins')
        print("\n")

        
    elif str(game[1]) == "X" and str(game[4])=="X" and str(game[7])== "X":
        print('X wins')
        print("\n")
    elif str(game[1]) == "O" and str(game[4])=="O" and str(game[7])== "O":
        print('O wins')
        print("\n")

        
    elif str(game[2]) == "X" and str(game[5])=="X" and str(game[8])== "X":
        print('X wins')
        print("\n")        
    elif str(game[2]) == "O" and str(game[5])=="O" and str(game[8])== "O":
        print('O wins')
        print("\n")
        
    else:
        if "-" in game:
            print("Gra w toku")
        else:
            print("Gra zakończona remisem")



        
        
# PRZESZUKIWANIE PRZESTRZENI: Lokalizacja planszy
# Licznik wykrytych plansz gry
object_counter = 0
# wczytaj
img = cv2.imread('anaconda3\Lib\site-packages\skimage\data\TicTacToe\Image1.jpg')  # wczytaj
# maskuj
image = maskImage(img)
# detekcja konturów
contoursMain, hierarchyMain = findingContours(image)
# sortowanie
sorted_contours = sorted(contoursMain, key=cv2.contourArea, reverse=True)
largest_item = sorted_contours[0]
# szukanie planszy
# pozwolenie na szukanie
search = True
gameboards = []
while search:
    # wykrycie kolejnych planszy gry
    if cv2.contourArea(sorted_contours[0]) > cv2.contourArea(largest_item) * 0.7:
        largest_item = sorted_contours[0]
        sorted_contours.pop(0)

        # przycięcie/ separacja planszy gry
        cropObject(largest_item, img)
        board = resizeImage(cv2.imread(
            "anaconda3\Lib\site-packages\skimage\data\TicTacToe\output_tictactoe" + str(object_counter) + ".jpg"))
        gameboards.append(board)
    else:
        # wykryty kontur jest mniejszy od konturu poszukiwanej planszy gry
        search = False

    # PRZESZUKIWANIE PRZESTRZENI: Rozpoznanie elementów planszy
# WYKONUJE SIĘ DLA KAŻDEJ WYKRYTEJ PLANSZY GRY Z OSOBNA
for play_num, board in enumerate(gameboards):
    # maska z obszaru planszy
    mask = maskImage(board)
    # detekcje obiektów na planszy
    contours, hierarchy = findingContours(mask)
    # sortowanie konturów
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    sorted_contours.pop(0)  # <- usunięcie Planszy z rozpatrywanych konturów (błędny znak x)

    contour_list = sieveContours(sorted_contours, hierarchy)
    contour_list.pop(0)  # <- usunięcie środkowego pola z rozpatrywanych konturów(błędny znak o)

    #wykrywanie planszy i pobieranie jej skrajnych punktów
    c = findingBoard(contours)
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])
    
    h,w, s = board.shape
    minx = 0
    maxx = w
    miny = 0
    maxy = h
    minmax = [minx, maxx, miny, maxy]
    minx = left[0]
    maxx = right[0]
    miny = top[1]
    maxy = bottom[1]
    minmax = [minx, maxx, miny, maxy]

    horizontal_diff = (maxx - minx) // 3
    vertical_diff = (maxy - miny) // 3

    #wyliczanie punktów przecięcia linii
    pointA = [minx + horizontal_diff, miny + vertical_diff]
    pointB = [minx + 2 * horizontal_diff, miny + vertical_diff]
    pointC = [minx + horizontal_diff, miny + 2 * vertical_diff]
    pointD = [minx + 2 * horizontal_diff, miny + 2 * vertical_diff]

    middle_points = []
    middle_points.append(pointA)
    middle_points.append(pointB)
    middle_points.append(pointC)
    middle_points.append(pointD)

    #detekcja i rozróżnienie figur i pustych pól
    circles, crosses = circlesCrosses(contour_list)
    tictactoe = circles + crosses
    centers_X = findCenter(crosses)
    centers_O = findCenter(circles)
    centers = centers_X + centers_O
    empty, xs, os = emptyCells(centers, middle_points, centers_X)
    centers_E = emptyCenters(empty, middle_points, minmax)

    #wypisanie na ekranie konsoli stanu gry
    
    # oznaczanie obiektów
    cv2.drawContours(image=board, contours=contour_list, contourIdx=-1,
                     color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    nameContour(centers_O, "O", board)
    nameContour(centers_X, "X", board)
    nameContour(centers_E, "-", board)
    
    gameState(xs, os)


# IMAGE OUTPUT
# dla każdej planszy gry
for i in range(0, len(gameboards)):
    cv2.imshow('Gameboard ' + str(i), gameboards[i])

cv2.waitKey(0)
cv2.destroyAllWindows()

Stan rozgrywki TicTacToe nr 1
-  |  O  |  -
---------------
-  |  -  |  -
---------------
X  |  -  |  X
Gra w toku
Stan rozgrywki TicTacToe nr 2
O  |  -  |  X
---------------
-  |  O  |  -
---------------
-  |  X  |  O
Wygrywa O!


Stan rozgrywki TicTacToe nr 3
-  |  -  |  O
---------------
O  |  X  |  -
---------------
-  |  X  |  -
Gra w toku
